[![AWS Data Wrangler](_static/logo.png "AWS Data Wrangler")](https://github.com/awslabs/aws-data-wrangler)

# 6 - Amazon Athena

[Wrangler](https://github.com/awslabs/aws-data-wrangler) has two ways to run queries on Athena and fetch the result as a DataFrame:

- **ctas_approach=True** (Default)

    Wraps the query with a CTAS and then reads the table data as parquet directly from s3.
    
    `PROS`: Faster and can handle some level of nested types.
    
    `CONS`: Requires create/delete table permissions on Glue and Does not support timestamp with time zone (A temporary table will be created and then deleted immediately).

- **ctas_approach=False**

    Does a regular query on Athena and parse the regular CSV result on s3.
    
    `PROS`: Does not require create/delete table permissions on Glue and supports timestamp with time zone.
    
    `CONS`: Slower (But stills faster than other libraries that uses the regular Athena API) and does not handle nested types at all.

In [1]:
import awswrangler as wr

## Enter your bucket name:

In [2]:
import getpass
bucket = getpass.getpass()
path = f"s3://{bucket}/data/"

 ··········································


### Creating a Parquet Table from the NOAA's CSV files

[Reference](https://registry.opendata.aws/noaa-ghcn/)

In [3]:
cols = ["id", "dt", "element", "value", "m_flag", "q_flag", "s_flag", "obs_time"]

df = wr.s3.read_csv(
    path="s3://noaa-ghcn-pds/csv/189",
    names=cols,
    parse_dates=["dt", "obs_time"])  # Read 10 files from the 1890 decade (~1GB)

df

,id,dt,element,value,m_flag,q_flag,s_flag,obs_time
0,ASN00070200,1890-01-01,PRCP,0,NaN,NaN,a,NaN
1,SF000782720,1890-01-01,PRCP,0,NaN,NaN,I,NaN
2,CA005022790,1890-01-01,TMAX,-222,NaN,NaN,C,NaN
3,CA005022790,1890-01-01,TMIN,-261,NaN,NaN,C,NaN
4,CA005022790,1890-01-01,PRCP,0,NaN,NaN,C,NaN
...,...,...,...,...,...,...,...,...
29240012,USC00181790,1899-12-31,PRCP,0,P,NaN,6,1830
29240013,ASN00061000,1899-12-31,PRCP,0,NaN,NaN,a,NaN
29240014,ASN00040284,1899-12-31,PRCP,0,NaN,NaN,a,NaN
29240015,ASN00048117,1899-12-31,PRCP,0,NaN,NaN,a,NaN


In [4]:
res = wr.s3.to_parquet(
    df=df,
    path=path,
    dataset=True,
    mode="overwrite",
    database="awswrangler_test",
    table="noaa"
)

In [5]:
wr.catalog.table(database="awswrangler_test", table="noaa")

,Column Name,Type,Partition,Comment
0,id,string,False,
1,dt,timestamp,False,
2,element,string,False,
3,value,bigint,False,
4,m_flag,string,False,
5,q_flag,string,False,
6,s_flag,string,False,
7,obs_time,string,False,


## Default reading (ctas_approach=True) 30x faster!

In [6]:
%%time

wr.athena.read_sql_query("SELECT * FROM noaa", database="awswrangler_test")

CPU times: user 19.5 s, sys: 5.36 s, total: 24.9 s
Wall time: 47.2 s


,id,dt,element,value,m_flag,q_flag,s_flag,obs_time
0,USC00440097,1893-10-11,PRCP,0,<NA>,<NA>,6,<NA>
1,ASN00046039,1893-10-11,PRCP,0,<NA>,<NA>,a,<NA>
2,USC00167040,1893-10-11,TMAX,328,<NA>,<NA>,6,<NA>
3,USC00167040,1893-10-11,TMIN,117,<NA>,<NA>,6,<NA>
4,USC00167040,1893-10-11,PRCP,0,<NA>,<NA>,6,<NA>
...,...,...,...,...,...,...,...,...
29240012,USC00303773,1899-12-31,SNOW,0,<NA>,<NA>,6,<NA>
29240013,USC00165090,1899-12-31,TMAX,100,<NA>,<NA>,6,<NA>
29240014,USC00165090,1899-12-31,TMIN,-33,<NA>,<NA>,6,<NA>
29240015,USC00165090,1899-12-31,PRCP,51,<NA>,<NA>,6,<NA>


## Reading with ctas_approach=False

In [7]:
%%time

wr.athena.read_sql_query("SELECT * FROM noaa", database="awswrangler_test", ctas_approach=False)

CPU times: user 8min 27s, sys: 5.77 s, total: 8min 33s
Wall time: 12min 21s


,id,dt,element,value,m_flag,q_flag,s_flag,obs_time
0,ASN00070200,1890-01-01,PRCP,0,<NA>,<NA>,a,<NA>
1,SF000782720,1890-01-01,PRCP,0,<NA>,<NA>,I,<NA>
2,CA005022790,1890-01-01,TMAX,-222,<NA>,<NA>,C,<NA>
3,CA005022790,1890-01-01,TMIN,-261,<NA>,<NA>,C,<NA>
4,CA005022790,1890-01-01,PRCP,0,<NA>,<NA>,C,<NA>
...,...,...,...,...,...,...,...,...
29240012,USC00395481,1899-12-31,SNOW,0,<NA>,<NA>,6,<NA>
29240013,ASN00063055,1899-12-31,PRCP,0,<NA>,<NA>,a,<NA>
29240014,USC00357814,1899-12-31,TMAX,78,<NA>,<NA>,6,<NA>
29240015,USC00357814,1899-12-31,TMIN,0,<NA>,<NA>,6,<NA>


## Batching (Good for restricted memory environments)

In [8]:
dfs = wr.athena.read_sql_query(
    "SELECT * FROM noaa",
    database="awswrangler_test",
    ctas_approach=True,
    chunksize=True  # Chunksize calculated automatically for ctas_approach.
)

for df in dfs:  # Batching
    print(len(df.index))

7559889
56320
9056256
3466240
9101312


In [9]:
dfs = wr.athena.read_sql_query(
    "SELECT * FROM noaa",
    database="awswrangler_test",
    ctas_approach=False,
    chunksize=10_000_000
)

for df in dfs:  # Batching
    print(len(df.index))

10000000
10000000
9240017


## Cleaning Up S3

In [10]:
wr.s3.delete_objects(path)

## Cleaning Up the Database

In [11]:
for table in wr.catalog.get_tables(database="awswrangler_test"):
    wr.catalog.delete_table_if_exists(database="awswrangler_test", table=table["Name"])